# **3.2 Nullspace of A: Solving $Ax = 0$ and $Rx = 0$**

- The nullspace $N(A)$ consists of all solutions to $Ax = 0$.
- Each solution vector $x \in \mathbb{R}^n$, so $N(A)$ is a subspace of $\mathbb{R}^n$.
- The column space $C(A)$ is a subspace of $\mathbb{R}^m$.

**Special Solution Example**

For  
$$
A = 
\begin{bmatrix}
1 & 2 \\
3 & 5
\end{bmatrix}
$$

A special solution to $As = 0$ is:
$$
s =
\begin{bmatrix}
-2 \\
1
\end{bmatrix}
$$

- Derived by setting the free variable $x_2 = 1$.
- The nullspace is all linear combinations of these special solutions.
- Workflow:
  1. Reduce $A$ to row-echelon form $R$.
  2. Extract special solutions from $Rx = 0$.

In [2]:
import numpy as np
from scipy.linalg import svd, null_space

def special_solutions(A, tol=1e-12):
    """
    Compute the special solution(s) to the homogeneous system Ax=0.
    """
    A = np.array(A, dtype=float)
    # Use SVD-based null_space computation
    null_sp = null_space(A, rcond=tol)
    
    if null_sp.size == 0:
        print("Only the trivial solution exists (x=0).")
        return np.zeros((A.shape[1], 1))
    
    return null_sp

A = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
])

special_sol = special_solutions(A)
print("Special solution(s) to Ax=0:\n", special_sol)

Special solution(s) to Ax=0:
 [[-0.40824829]
 [ 0.81649658]
 [-0.40824829]]


### **Pivot Columns and Free Columns**

- Free variables correspond to columns **without pivots**.
- Special solutions: pick one free variable = 1, the rest = 0.
- Example: A $1 \times 3$ matrix with 1 pivot + 2 free vars yields two special solutions:
  - $(x_2 = 0, x_3 = 1)$
  - $(x_2 = 1, x_3 = 0)$

In [3]:
import numpy as np

def pivot_and_free_columns(A, tol=1e-12):
    """
    Identify pivot columns and free columns of matrix A using Gaussian elimination.
    
    Parameters:
        A (np.ndarray): m x n matrix
        tol (float): Tolerance for detecting non-zero pivot
    """
    A = np.array(A, dtype=float)
    m, n = A.shape
    A_rref = A.copy()
    pivot_columns = []
    
    row = 0
    for col in range(n):
        # Find pivot in this column
        pivot_rows = np.where(np.abs(A_rref[row:, col]) > tol)[0]
        if len(pivot_rows) == 0:
            continue
        pivot_row = pivot_rows[0] + row
        
        # Swap current row with pivot_row
        if pivot_row != row:
            A_rref[[row, pivot_row], :] = A_rref[[pivot_row, row], :]
        
        # Normalize pivot row
        A_rref[row, :] = A_rref[row, :] / A_rref[row, col]
        pivot_columns.append(col)
        
        # Eliminate below
        for r in range(row+1, m):
            A_rref[r, :] -= A_rref[r, col] * A_rref[row, :]
        
        row += 1
        if row >= m:
            break
    
    free_columns = [c for c in range(n) if c not in pivot_columns]
    return {"pivot_columns": pivot_columns, "free_columns": free_columns}

A = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
])

result = pivot_and_free_columns(A)
print("Pivot columns:", result["pivot_columns"])
print("Free columns:", result["free_columns"])

Pivot columns: [0, 1]
Free columns: [2]


### **Reduced Row Echelon Form (RREF)**

- For rectangular $A$, elimination continues past $U$ into full reduction.
- RREF steps:
  1. Zero out entries *above* pivots.
  2. Normalize pivot rows so pivots become 1.

**Notes**

- Many matrices have only the trivial solution $x = 0$.
- Nullspace $N(A) = \{0\}$ → no special solutions → columns are independent.
- Counts:
  - number of free vars = count of special solutions.
  - number of pivots = rank.
- If $n > m$, then $Ax = 0$ has at least one nonzero solution (at least one free column).


In [5]:
import numpy as np

def rref(A, tol=1e-12):
    """
    Convert a matrix A to its Reduced Row Echelon Form (RREF).

    Parameters:
        A (np.ndarray): Input m x n matrix
        tol (float): Tolerance for detecting non-zero pivots
    """
    A = np.array(A, dtype=float)
    m, n = A.shape
    R = A.copy()
    row = 0

    for col in range(n):
        # Find pivot in column
        pivot_rows = np.where(np.abs(R[row:, col]) > tol)[0]
        if len(pivot_rows) == 0:
            continue
        pivot_row = pivot_rows[0] + row

        # Swap pivot row with current row
        if pivot_row != row:
            R[[row, pivot_row], :] = R[[pivot_row, row], :]

        # Normalize pivot row
        R[row, :] = R[row, :] / R[row, col]

        # Eliminate all other entries in this column
        for r in range(m):
            if r != row:
                R[r, :] -= R[r, col] * R[row, :]

        row += 1
        if row >= m:
            break

    # Set near-zero entries to exact zero
    R[np.abs(R) < tol] = 0
    return R

A = np.array([
    [1, 2, -1, -4],
    [2, 3, -1, -11],
    [-2, 0, -3, 22]
])

R = rref(A)
print("RREF of A:\n", R)

RREF of A:
 [[ 1.  0.  0. -8.]
 [ 0.  1.  0.  1.]
 [ 0.  0.  1. -2.]]


### **Rank of a Matrix**

- Definition: $\text{rank}(A) = r =$ number of pivots.
- Every free column is a linear combination of earlier pivot columns.
- Special solutions encode those combinations.

In [6]:
import numpy as np

def matrix_rank(A, tol=1e-12):
    """
    Compute the rank of a matrix A.
    """
    A = np.array(A, dtype=float)
    s = np.linalg.svd(A, compute_uv=False)  # Singular values
    rank = np.sum(s > tol)
    return rank

A = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
])

print("Rank of A:", matrix_rank(A))

Rank of A: 2


### **Rank-One Matrices**

- Only one pivot.
- All rows become multiples of the pivot row; all columns become multiples of the pivot column.
- Column space is 1-dimensional.

For rank 1:
$$
Ax = u(v^T x) = 0 \Rightarrow v^T x = 0
$$
→ The nullspace is the set of vectors orthogonal to $v$.  
Row space = line; nullspace = perpendicular line.

In [7]:
import numpy as np

def rank_one_subspaces(A, tol=1e-12):
    """
    Analyze a rank-1 matrix A and return:
    - Column space (1D)
    - Row space (1D)
    - Nullspace (vectors orthogonal to v)
    """
    A = np.array(A, dtype=float)
    m, n = A.shape

    # Identify a non-zero row and column (pivot)
    row_norms = np.linalg.norm(A, axis=1)
    col_norms = np.linalg.norm(A, axis=0)

    pivot_row_idx = np.argmax(row_norms)
    pivot_col_idx = np.argmax(col_norms)

    u = A[:, pivot_col_idx]  # Column vector spanning column space
    v = A[pivot_row_idx, :]  # Row vector spanning row space

    # Normalize
    if np.linalg.norm(u) > tol:
        u = u / np.linalg.norm(u)
    if np.linalg.norm(v) > tol:
        v = v / np.linalg.norm(v)

    # Nullspace: all vectors x orthogonal to v => v^T x = 0
    # Generate basis for nullspace
    from scipy.linalg import null_space
    nullspace_vectors = null_space(v.reshape(1, -1), rcond=tol)

    return {
        "column_space": u,
        "row_space": v,
        "nullspace": nullspace_vectors
    }

A = np.array([
    [2, 4, 6],
    [1, 2, 3],
    [3, 6, 9]
])

subspaces = rank_one_subspaces(A)
print("Column space vector:\n", subspaces["column_space"])
print("Row space vector:\n", subspaces["row_space"])
print("Nullspace basis vectors:\n", subspaces["nullspace"])

Column space vector:
 [0.53452248 0.26726124 0.80178373]
Row space vector:
 [0.26726124 0.53452248 0.80178373]
Nullspace basis vectors:
 [[-0.53452248 -0.80178373]
 [ 0.77454192 -0.33818712]
 [-0.33818712  0.49271932]]


**Key Ideas**

1. $N(A)$ is a subspace of $\mathbb{R}^n$ containing all solutions to $Ax = 0$.
2. Elimination produces $rref(A)$ with pivot and free columns.
3. Each free variable yields a special solution (set it to 1, others to 0).
4. Rank $r$ = number of pivots.
5. Full solution to $Ax = 0$ = combination of $n - r$ special solutions.
6. If $n > m$, a free column must exist → nonzero solution to $Ax = 0$.